In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import random as rn
import csv
import os
import cv2
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
# tensorflowの乱数シード固定(再現性の担保)
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
batch_size = 32
n_epochs = 50

# 画像データ取り込み

In [ ]:
DATA_FOLDER = '../data/'
datalist = pd.read_csv(os.path.join(DATA_FOLDER, 'frames_data.csv'), names=["img_path", "l_class", 's_class','timestamp'])
datalist.head()

In [ ]:
l_classlist = ['4gokan', '5gokan', '5gokan-denkisogokan', '5gokan-mediacenter'
                    '8gokan', 'denkisogokan', 'lab', 'lab-lounge', 'lounge']
s_classlist = ['4gokan_inside', '4gokan_outside', '5gokan-denkisogokan', '5gokan-mediacenter',
'5gokan_1F', '5gokan_2F', '5gokan_3F', '5gokan_ent_east', '5gokan_lounge',
'5gokan_out_east', '5gokan_parking', '5gokan_smoking', '5gokan_stairs_cnt',
'5gokan_stairs_west', '8gokan_1F', '8gokan_ent_north', '8gokan_ent_south',
'denkisogokan_2F', 'denkisogokan_3F', 'denkisogokan_4F', 'denkisogokan_elevator',
'denkisogokan_lounge', 'denkisogokan_stairs', 'lab-lounge', 'lab_bs_cnt',
'lab_corner', 'lab_desk', 'lab_desk_table', 'lab_ent', 'lab_printer',
'lab_table', 'lab_wb_cnt', 'lab_wb_ent', 'lounge']
num_l_class =len(l_classlist)
num_s_class = len(s_classlist)

In [ ]:
dfl = datalist.drop(['s_class','timestamp'], axis=1)
dfl.head()

In [ ]:
dfl.groupby('l_class').count()

In [ ]:
dfs = datalist.drop(['l_class', 'timestamp'], axis=1)
dfs.groupby('s_class').count()

In [ ]:
grad_test_img_path =[
    "../data/insta_frames/4gokan/4gokan_inside/4gokan_inside_0_img_000240.png",
    "../data/insta_frames/4gokan/4gokan_outside/4gokan_outside_0_img_000180.png",
    "../data/insta_frames/5gokan-denkisogokan/5gokan-denkisogokan_1_img_000010.png",
    "../data/insta_frames/5gokan-mediacenter/5gokan-mediacenter_0_img_000030.png",
    "../data/insta_frames/5gokan/5gokan_1F/5gokan_1F_0_img_000070.png",
    "../data/insta_frames/5gokan/5gokan_2F/5gokan_2F_2_img_000030.png",
    "../data/insta_frames/5gokan/5gokan_3F/5gokan_3F_0_img_000040.png",
    "../data/insta_frames/5gokan/5gokan_ent_east/5gokan_ent_east_2_img_000000.png",
    "../data/insta_frames/5gokan/5gokan_lounge/5gokan_lounge_2_img_000060.png",
    "../data/insta_frames/5gokan/5gokan_out_east/5gokan_out_east_0_img_000270.png",
    "../data/insta_frames/5gokan/5gokan_parking/5gokan_parking_1_img_000170.png",
    "../data/insta_frames/5gokan/5gokan_smoking/5gokan_smoking_0_img_000270.png",
    "../data/insta_frames/5gokan/5gokan_stairs_cnt/5gokan_stairs_cnt_3_img_000010.png",
    "../data/insta_frames/5gokan/5gokan_stairs_west/5gokan_stairs_west_0_img_000340.png",
    "../data/insta_frames/8gokan/8gokan_1F/8gokan_1F_1_img_000000.png",
    "../data/insta_frames/8gokan/8gokan_ent_north/8gokan_ent_north_0_img_000010.png",
    "../data/insta_frames/8gokan/8gokan_ent_south/8gokan_ent_south_1_img_000070.png",
    "../data/insta_frames/denkisogokan/denkisogokan_2F/denkisogokan_2F_2_img_000300.png",
    "../data/insta_frames/denkisogokan/denkisogokan_3F/denkisogokan_3F_0_img_000010.png",
    "../data/insta_frames/denkisogokan/denkisogokan_4F/denkisogokan_4F_0_img_000170.png",
    "../data/insta_frames/denkisogokan/denkisogokan_elevator/denkisogokan_elevator_0_img_000220.png",
    "../data/insta_frames/denkisogokan/denkisogokan_lounge/denkisogokan_lounge_1_img_000060.png",
    "../data/insta_frames/denkisogokan/denkisogokan_stairs/denkisogokan_stairs_0_img_000120.png",
    "../data/insta_frames/lab-lounge/lab-lounge_0_img_000170.png",
    "../data/insta_frames/lab/lab_bs_cnt/lab_bs_cnt_0_img_000270.png",
    "../data/insta_frames/lab/lab_corner/lab_corner_0_img_000270.png",
    "../data/insta_frames/lab/lab_desk/lab_desk_8_img_000210.png",
    "../data/insta_frames/lab/lab_desk_table/lab_desk_table_3_img_000150.png",
    "../data/insta_frames/lab/lab_ent/lab_ent_0_img_000020.png",
    "../data/insta_frames/lab/lab_printer/lab_printer_0_img_000030.png",
    "../data/insta_frames/lab/lab_table/lab_table_0_img_000170.png",
    "../data/insta_frames/lab/lab_wb_cnt/lab_wb_cnt_0_img_000120.png",
    "../data/insta_frames/lab/lab_wb_ent/lab_wb_ent_1_img_000160.png",
    "../data/insta_frames/lounge/lounge_1_img_000060.png",
]

# Markdown
from sklearn.preprocessing import LabelEncoder

# Markdown
le = LabelEncoder()
le.fit(dfs.s_class)
dfs["labels"] = le.transform(dfs.s_class)
dfs.groupby('labels')

# Markdown
cor_table = dfs.groupby('labels').s_class.unique()
cor_table = pd.DataFrame(cor_table)
cor_table.head()

# Markdown
cor_table.to_csv('cor_table.csv', index=False)

# Markdown
dfs = dfs.drop(['s_class'], axis=1)

# Markdown
x = []
for i in range(len(dfs.img_path)):
    image = img_to_array(load_img(dfs.img_path[i], target_size=(224,224)))
    x.append(image)

# Markdown
y = dfs.labels
print(y)

# Markdown
#正規化
x = np.array(x, dtype='float32')/255

# Markdown
#np.array形式のデータを保存　x, y
np.save(file='x.npy', arr=x)
np.save(file='y.npy', arr=y)

In [ ]:
#x, yをload
x = np.load(file='x.npy')
y = np.load(file='y.npy')

In [ ]:
print(x.shape)
print(y.shape)

# train test & validation set split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x, x_valid, y, y_valid = train_test_split(x,y, random_state=42, test_size=0.2, stratify=y)

# EDA(探索的データ解析)

In [ ]:
fig = plt.figure(figsize=(9,18))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)
for i in range(5):
    ax = fig.add_subplot(1, 5, i+1, xticks=[], yticks=[])
    ax.imshow(x[i])

# CNN(keras NotPretrained)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Activation, add, Add, Dropout, BatchNormalization, GlobalMaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
inputs = Input(shape=(224,224,3))
x1 = Conv2D(64, kernel_size=(3,3), activation='relu')(inputs)
x2 = Conv2D(64, kernel_size=(3,3), activation='relu')(x1)
x3 = MaxPooling2D(pool_size=(2,2))(x2)
x4 = Conv2D(128, kernel_size=(3,3), activation='relu')(x3)
x5 = Conv2D(128, kernel_size=(3,3), activation='relu')(x4)
x6 = MaxPooling2D(pool_size=(2,2))(x5)
x7 = Flatten()(x6)
prediction = Dense(num_s_class, activation='softmax')(x7)

model = Model(inputs=inputs, outputs=prediction)
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, mode='auto')
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
feature1 = GlobalMaxPooling2D(name='Conv2D_1')(x1)
feature2 = GlobalMaxPooling2D(name='Conv2D_1')(x2)
feature3 = GlobalMaxPooling2D(name='Conv2D_1')(x3)
feature4 = GlobalMaxPooling2D(name='Conv2D_1')(x4)
feature5 = GlobalMaxPooling2D(name='Conv2D_1')(x5)
feature6 = GlobalMaxPooling2D(name='Conv2D_1')(x6)

In [ ]:
hidden_model1 = Model(inputs=model.input, outputs=feature1)
hidden_model2 = Model(inputs=model.input, outputs=feature2)
hidden_model3 = Model(inputs=model.input, outputs=feature3)
hidden_model4 = Model(inputs=model.input, outputs=feature4)
hidden_model5 = Model(inputs=model.input, outputs=feature5)
hidden_model6= Model(inputs=model.input, outputs=feature6)

   # 層化抽出法(Stratified sampling)&KFold

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.utils import np_utils

In [ ]:
y.shape

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=42)
cvscores = []
#Stratified KFoldはlabel encoding化した整数ベクトルでないともちいることができない
x = np.array(x)
y = np.array(y)
for train_index, test_index in skf.split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #One-hot化　CNNに入出力できるように整形
    y_train = np_utils.to_categorical(y_train, num_s_class)
    y_test = np_utils.to_categorical(y_test, num_s_class)
    model.fit(x_train, y_train, verbose=1, batch_size=batch_size, epochs=n_epochs,
              callbacks=[early_stopping], validation_split=0.2)
    scores = model.evaluate(x_test, y_test)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [ ]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

→ validation_dataを用意しないとval_lossを計算できない

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from scipy.sparse.csgraph import connected_components

In [ ]:
y_pred = model.predict(x_valid)

In [ ]:
y_valid = np_utils.to_categorical(y_valid)

In [ ]:
y_pred = np.array(y_pred)
y_true = np.array(y_valid)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

In [ ]:
print(y_pred.shape)
print(y_true.shape)

In [ ]:
cmx = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12,12))
sns.heatmap(cmx, annot=True)
plt.show()
print('accuracy: {}'.format(accuracy_score
(y_true, y_pred)))

In [ ]:
cor_table = pd.read_csv('cor_table.csv')
print(cor_table)

# 中間層特徴量の可視化

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
hidden1 = hidden_model1.predict(x_train)
hidden2 = hidden_model2.predict(x_train)
hidden3 = hidden_model3.predict(x_train)
hidden4 = hidden_model4.predict(x_train)
hidden5 = hidden_model5.predict(x_train)
hidden6 = hidden_model6.predict(x_train)

In [ ]:
print(hidden1.shape)
print(hidden2.shape)
print(hidden3.shape)
print(hidden4.shape)
print(hidden5.shape)
print(hidden6.shape)

#標準化
scaler = StandardScaler()
scaler.fit(hidden1)
hidden1 = scaler.transform(hidden1)
scaler.fit(hidden2)
hidden1 = scaler.transform(hidden2)
scaler.fit(hidden3)
hidden1 = scaler.transform(hidden3)
scaler.fit(hidden4)
hidden1 = scaler.transform(hidden4)
scaler.fit(hidden5)
hidden1 = scaler.transform(hidden5)
scaler.fit(hidden6)
hidden1 = scaler.transform(hidden6)

In [ ]:
pca = PCA()

In [ ]:
cm_name = 'jet'
cmap = plt.get_cmap(cm_name, num_s_class)

In [ ]:
# one-hotから整数値ベクトルに変換
y_train_vec = []
for i in range(len(y_train)):
    y_train_vec.append(np.argmax(y_train[i]))

In [ ]:
feature1 = pca.fit_transform(hidden1)
#　主成分分析の例(1層目)
df_feature1 = pd.DataFrame(feature1)
print(df_feature1)

In [ ]:
plt.scatter(feature1[:,0],feature1[:,1], marker=".", c=y_train_vec, cmap=cmap)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar()
plt.show()

In [ ]:
pca.fit(hidden1)
ev_ratio = pca.explained_variance_ratio_
ev_ratio = np.hstack([0, ev_ratio.cumsum()])
plt.xlabel("num of components")
plt.ylabel("explained variance ratio")
plt.plot(ev_ratio)
plt.show()

# 2層目の特徴ベクトル (Conv2D)

In [ ]:
feature2 = pca.fit_transform(hidden2)

In [ ]:
plt.scatter(feature2[:,0],feature2[:,1], marker=".", c=y_train_vec, cmap=cmap)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar()
plt.show()

In [ ]:
pca.fit(hidden2)
ev_ratio = pca.explained_variance_ratio_
ev_ratio = np.hstack([0, ev_ratio.cumsum()])
plt.xlabel("num of components")
plt.ylabel("explained variance ratio")
plt.plot(ev_ratio)
plt.show()

# 3層目の特徴ベクトル (MaxPooling2D)

In [ ]:
feature3 = pca.fit_transform(hidden3)

In [ ]:
plt.scatter(feature3[:,0],feature3[:,1], marker=".", c=y_train_vec, cmap=cmap)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar()
plt.show()

In [ ]:
pca.fit(hidden3)
ev_ratio = pca.explained_variance_ratio_
ev_ratio = np.hstack([0, ev_ratio.cumsum()])
plt.xlabel("num of components")
plt.ylabel("explained variance ratio")
plt.plot(ev_ratio)
plt.show()

# 4層目の特徴ベクトル (Conv2D)

In [ ]:
feature4 = pca.fit_transform(hidden4)

In [ ]:
plt.scatter(feature4[:,0],feature4[:,1], marker=".", c=y_train_vec, cmap=cmap)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar()
plt.show()

In [ ]:
pca.fit(hidden4)
ev_ratio = pca.explained_variance_ratio_
ev_ratio = np.hstack([0, ev_ratio.cumsum()])
plt.xlabel("num of components")
plt.ylabel("explained variance ratio")
plt.plot(ev_ratio)
plt.show()

# 5層目の特徴ベクトル (Conv2D)

In [ ]:
feature5 = pca.fit_transform(hidden5)

In [ ]:
plt.scatter(feature5[:,0],feature5[:,1], marker=".", c=y_train_vec, cmap=cmap)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar()
plt.show()

In [ ]:
pca.fit(hidden5)
ev_ratio = pca.explained_variance_ratio_
ev_ratio = np.hstack([0, ev_ratio.cumsum()])
plt.xlabel("num of components")
plt.ylabel("explained variance ratio")
plt.plot(ev_ratio)
plt.show()

# 6層目の特徴ベクトル(MaxPooling2D)

In [ ]:
feature6 = pca.fit_transform(hidden6)

In [ ]:
plt.scatter(feature6[:,0],feature6[:,1], marker=".", c=y_train_vec, cmap=cmap)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar()
plt.show()

In [ ]:
pca.fit(hidden6)
ev_ratio = pca.explained_variance_ratio_
ev_ratio = np.hstack([0, ev_ratio.cumsum()])
plt.xlabel("num of components")
plt.ylabel("explained variance ratio")
plt.plot(ev_ratio)
plt.show()

# grad-cam

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
true = []
for i in range(num_s_class):
    true.append(i)

In [ ]:
pred = []

grad_last_conv = model.get_layer("conv2d_3")
model_output = model.output[:,0]
print(grad_last_conv)
print(model_output)
grads = K.gradients(model_output, grad_last_conv.output)[0]
pooled_grads = K.mean(grads, axis=(0,1,2))
iterate = K.function([model.input], [pooled_grads, grad_last_conv.output[0]])

for n in range(len(grad_test_img_path)):
    img_keras = image.load_img(grad_test_img_path[n], target_size=(224, 224))
    img_tensor = image.img_to_array(img_keras)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    predicts = model.predict(img_tensor)
    s_class_num = np.argmax(predicts)
    pred.append(s_class_num)
    img_tensor /= 255.
    pooled_grads_val, conv_output_val = iterate([img_tensor])
    for i in range(pooled_grads_val.shape[0]):
        conv_output_val[:, :, i] *= pooled_grads_val[i]
    heatmap = np.mean(conv_output_val, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    img = cv2.imread(grad_test_img_path[n])
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    save_dir = '../data/gradcam_img/selfCNN/'+s_classlist[n]
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)	# Make a directory
    #保存先ディレクトリ名はその画像のクラス，画像の予測値を画像の名前に書き込む
    cv2.imwrite(save_dir+'/heatmap_pred_'+s_classlist[s_class_num]+'.jpg', superimposed_img)
    print('Saved: ', save_dir+'/heatmap_pred_'+s_classlist[s_class_num]+'.jpg')

In [ ]:
cmx = confusion_matrix(true, pred)
plt.figure(figsize=(12,12))
sns.heatmap(cmx, annot=True)
plt.ylabel("truth")
plt.xlabel("prediction")
plt.show()

In [ ]:
cor_table = pd.read_csv('cor_table.csv')
print(cor_table)